# Day 16: Supervised Fine-Tuning (SFT) Implementation

In this notebook, we'll implement Supervised Fine-Tuning (SFT) on a small pre-trained language model. We'll use a subset of instruction data to fine-tune the model to follow instructions and generate helpful responses.

## Overview

1. Load a small pre-trained model
2. Prepare instruction dataset
3. Format data for training
4. Implement SFT training loop
5. Evaluate the fine-tuned model
6. Compare pre-trained vs. fine-tuned outputs

In [ ]:
# Import necessary libraries
import torch
import numpy as np
import pandas as pd
import json
import os
import random
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from peft import get_peft_model, LoraConfig, TaskType
from tqdm import tqdm

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## 1. Load a Small Pre-trained Model

We'll use a small pre-trained model for this demonstration. In a real-world scenario, you might use larger models like Llama, Mistral, or GPT-J.

In [ ]:
# For this example, we'll use a small model to make it runnable on most hardware
model_name = "gpt2"  # ~124M parameters

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Add padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

print(f"Model loaded: {model_name}")
print(f"Number of parameters: {model.num_parameters():,}")

## 2. Prepare Instruction Dataset

We'll create a small synthetic instruction dataset for demonstration purposes. In practice, you would use datasets like Stanford Alpaca, Dolly, or FLAN.

In [ ]:
# Create a small synthetic instruction dataset
synthetic_instructions = [
    {
        "instruction": "Explain the concept of machine learning in simple terms.",
        "input": "",
        "output": "Machine learning is when computers learn from examples instead of being explicitly programmed. It's like teaching a child to recognize dogs by showing them many pictures of dogs, rather than listing all the features that make a dog a dog."
    },
    {
        "instruction": "Write a short poem about the ocean.",
        "input": "",
        "output": "Vast blue expanse beneath the sky,\nWaves dance and crash as time goes by.\nSecrets deep in waters cold,\nOcean stories, forever told."
    },
    {
        "instruction": "Summarize the following paragraph.",
        "input": "Artificial intelligence has made significant strides in recent years, transforming industries from healthcare to finance. Machine learning algorithms now power recommendation systems, autonomous vehicles, and medical diagnostic tools. Despite these advances, challenges remain in areas such as bias mitigation, explainability, and ensuring AI systems align with human values.",
        "output": "AI has advanced rapidly, impacting healthcare and finance through applications like recommendation systems and autonomous vehicles, though challenges persist in bias, explainability, and alignment with human values."
    },
    {
        "instruction": "Translate the following English text to French.",
        "input": "Hello, how are you today? I hope you're doing well.",
        "output": "Bonjour, comment allez-vous aujourd'hui ? J'espère que vous allez bien."
    },
    {
        "instruction": "Explain the difference between RAM and ROM.",
        "input": "",
        "output": "RAM (Random Access Memory) is temporary, volatile memory that stores data a computer is actively using. It's erased when the computer is turned off. ROM (Read-Only Memory) is permanent, non-volatile memory that stores instructions needed to start up a computer. It retains data even when the computer is turned off."
    },
    {
        "instruction": "Generate a list of 5 creative uses for a paperclip.",
        "input": "",
        "output": "1. Use as a tiny bookmark for specific lines in a book\n2. Create miniature sculptures or art pieces\n3. Use as a makeshift phone stand by bending it appropriately\n4. Make a simple hook for hanging small items\n5. Use as an emergency zipper pull replacement"
    },
    {
        "instruction": "Explain the concept of photosynthesis.",
        "input": "",
        "output": "Photosynthesis is the process by which plants, algae, and some bacteria convert sunlight, water, and carbon dioxide into glucose (sugar) and oxygen. The plants use the glucose as food for energy and growth, while the oxygen is released into the atmosphere. This process is essential for life on Earth as it produces oxygen and forms the base of most food chains."
    },
    {
        "instruction": "Provide three tips for effective time management.",
        "input": "",
        "output": "1. Prioritize tasks using methods like the Eisenhower Matrix (urgent/important)\n2. Break large projects into smaller, manageable tasks with specific deadlines\n3. Use the Pomodoro Technique: work in focused 25-minute intervals followed by short breaks"
    },
    {
        "instruction": "Calculate the area of a circle with radius 5 cm.",
        "input": "",
        "output": "To calculate the area of a circle, I use the formula: Area = π × r². With a radius of 5 cm:\nArea = π × 5² = π × 25 = 78.54 cm²"
    },
    {
        "instruction": "Explain the difference between weather and climate.",
        "input": "",
        "output": "Weather refers to short-term atmospheric conditions in a specific place and time, such as temperature, humidity, precipitation, and wind. Climate, on the other hand, describes the long-term patterns of weather in a particular region, typically averaged over a period of 30 years or more. Simply put, weather is what you experience day to day, while climate is the average weather pattern of a region over many years."
    }
]

# Convert to DataFrame for easier manipulation
df = pd.DataFrame(synthetic_instructions)
print(f"Dataset size: {len(df)} examples")
df.head(3)

### Option: Load a Real Instruction Dataset

Alternatively, you can load a real instruction dataset from the Hugging Face Hub. Uncomment the code below to use the Stanford Alpaca dataset.

In [ ]:
# # Load a subset of the Stanford Alpaca dataset
# try:
#     dataset = load_dataset("tatsu-lab/alpaca", split="train[:100]")
#     df = pd.DataFrame({
#         "instruction": dataset["instruction"],
#         "input": dataset["input"],
#         "output": dataset["output"]
#     })
#     print(f"Loaded {len(df)} examples from Stanford Alpaca dataset")
# except Exception as e:
#     print(f"Error loading dataset: {e}")
#     print("Using synthetic dataset instead")

## 3. Format Data for Training

We need to format our instruction data for the model. We'll use a template that combines the instruction, input, and output.

In [ ]:
def format_instruction(row):
    """Format instruction data into a single string."""
    instruction = row["instruction"]
    input_text = row["input"]
    output = row["output"]
    
    if input_text:
        formatted = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n{output}"
    else:
        formatted = f"### Instruction:\n{instruction}\n\n### Response:\n{output}"
    
    return formatted

# Apply formatting to our dataset
df["formatted_text"] = df.apply(format_instruction, axis=1)

# Display an example
print("Example of formatted instruction:")
print("\n" + "-"*50 + "\n")
print(df["formatted_text"].iloc[2])
print("\n" + "-"*50)

## 4. Tokenize and Prepare Dataset for Training

In [ ]:
# Split into train and validation sets
train_df = df.sample(frac=0.8, random_state=42)
val_df = df.drop(train_df.index)

print(f"Training examples: {len(train_df)}")
print(f"Validation examples: {len(val_df)}")

# Convert to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df[["formatted_text"]])
val_dataset = Dataset.from_pandas(val_df[["formatted_text"]])

# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples["formatted_text"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

# Tokenize datasets
tokenized_train_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["formatted_text"]
)

tokenized_val_dataset = val_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["formatted_text"]
)

# Add labels for causal language modeling (labels are the same as input_ids)
tokenized_train_dataset = tokenized_train_dataset.map(lambda examples: {"labels": examples["input_ids"]})
tokenized_val_dataset = tokenized_val_dataset.map(lambda examples: {"labels": examples["input_ids"]})

## 5. Set Up LoRA for Parameter-Efficient Fine-Tuning

Instead of fine-tuning all parameters, we'll use LoRA (Low-Rank Adaptation) to make the process more efficient.

In [ ]:
# Configure LoRA
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,  # Rank of the update matrices
    lora_alpha=32,  # Scaling factor
    lora_dropout=0.1,  # Dropout probability for LoRA layers
    target_modules=["c_attn", "c_proj"]  # Layers to apply LoRA to (specific to GPT-2)
)

# Create LoRA model
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

## 6. Set Up Training Arguments and Trainer

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    load_best_model_at_end=True,
    learning_rate=5e-5,
    fp16=torch.cuda.is_available(),  # Use mixed precision if available
    gradient_accumulation_steps=2,  # Accumulate gradients over 2 steps
    report_to="none"  # Disable reporting to avoid dependencies
)

# Create data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # We're doing causal language modeling, not masked language modeling
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    data_collator=data_collator
)

## 7. Train the Model

Now we'll fine-tune the model on our instruction dataset.

In [ ]:
# Train the model
trainer.train()

## 8. Evaluate the Model

Let's compare the outputs of the pre-trained model and our fine-tuned model.

In [ ]:
# Save the fine-tuned model
model_path = "./fine_tuned_model"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

# Load the original pre-trained model for comparison
original_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
original_model.eval()

# Load the fine-tuned model
fine_tuned_model = model.to(device)
fine_tuned_model.eval()

In [ ]:
def generate_response(model, instruction, input_text=""):
    """Generate a response from the model given an instruction and optional input."""
    # Format the prompt
    if input_text:
        prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n"
    else:
        prompt = f"### Instruction:\n{instruction}\n\n### Response:\n"
    
    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    # Generate response
    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            max_new_tokens=100,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id
        )
    
    # Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract just the response part
    response = response.split("### Response:")[-1].strip()
    
    return response

# Test instructions
test_instructions = [
    {"instruction": "Explain what a neural network is in simple terms.", "input": ""},
    {"instruction": "Write a short poem about mountains.", "input": ""},
    {"instruction": "Summarize the following text.", "input": "Climate change is the long-term alteration of temperature and typical weather patterns in a place. It could refer to a particular location or the planet as a whole. Climate change has been connected to damaging weather events such as more frequent and more intense hurricanes, floods, downpours, and winter storms."}
]

# Compare original and fine-tuned model outputs
for test in test_instructions:
    print(f"\n{'='*80}\n")
    print(f"Instruction: {test['instruction']}")
    if test['input']:
        print(f"Input: {test['input']}")
    
    print("\nOriginal Model Response:")
    original_response = generate_response(original_model, test['instruction'], test['input'])
    print(original_response)
    
    print("\nFine-tuned Model Response:")
    fine_tuned_response = generate_response(fine_tuned_model, test['instruction'], test['input'])
    print(fine_tuned_response)

## 9. Advanced: Evaluating Instruction Following

Let's evaluate how well our model follows instructions using a simple scoring function.

In [ ]:
def evaluate_instruction_following(model, test_cases):
    """Evaluate how well the model follows instructions."""
    scores = []
    
    for i, test in enumerate(test_cases):
        instruction = test["instruction"]
        input_text = test.get("input", "")
        expected_type = test["expected_type"]
        
        # Generate response
        response = generate_response(model, instruction, input_text)
        
        # Score based on expected response type
        score = 0
        if expected_type == "explanation" and len(response.split()) > 20:
            score = 1  # Basic check for explanation length
        elif expected_type == "poem" and "\n" in response:
            score = 1  # Basic check for poem format (line breaks)
        elif expected_type == "summary" and len(response.split()) < len(input_text.split()) * 0.7:
            score = 1  # Basic check for summary (shorter than input)
        
        scores.append(score)
        
        print(f"Test {i+1}: {'Passed' if score == 1 else 'Failed'}")
        print(f"Instruction: {instruction}")
        if input_text:
            print(f"Input: {input_text[:50]}...")
        print(f"Response: {response[:100]}..." if len(response) > 100 else f"Response: {response}")
        print("-" * 50)
    
    avg_score = sum(scores) / len(scores) if scores else 0
    print(f"\nAverage instruction following score: {avg_score:.2f} ({sum(scores)}/{len(scores)} tests passed)")
    
    return avg_score

# Test cases with expected response types
test_cases = [
    {"instruction": "Explain what a neural network is in simple terms.", "input": "", "expected_type": "explanation"},
    {"instruction": "Write a short poem about mountains.", "input": "", "expected_type": "poem"},
    {"instruction": "Summarize the following text.", "input": "Climate change is the long-term alteration of temperature and typical weather patterns in a place. It could refer to a particular location or the planet as a whole. Climate change has been connected to damaging weather events such as more frequent and more intense hurricanes, floods, downpours, and winter storms.", "expected_type": "summary"}
]

print("Evaluating original model:")
original_score = evaluate_instruction_following(original_model, test_cases)

print("\nEvaluating fine-tuned model:")
fine_tuned_score = evaluate_instruction_following(fine_tuned_model, test_cases)

print(f"\nImprovement: {(fine_tuned_score - original_score) * 100:.1f}%")

## 10. Conclusion

In this notebook, we've implemented Supervised Fine-Tuning (SFT) on a small pre-trained language model. We've seen how SFT can improve a model's ability to follow instructions and generate helpful responses.

Key takeaways:
1. SFT is a straightforward but powerful technique for aligning language models with human intent
2. Even with a small dataset, we can see improvements in instruction following
3. Parameter-efficient fine-tuning methods like LoRA make SFT feasible on consumer hardware
4. The quality of the instruction dataset is crucial for effective SFT

Next steps:
- Experiment with larger instruction datasets
- Try different prompt formats
- Explore more advanced alignment techniques like RLHF and DPO